# Projective Simulation Tutorial



## a little bit of theory

Am not afraid of this. There are just a few things you should understand basically bevore we can start with practice. So let's begin: <br>
At first we should understand the model of Reinforcement Learning (RL), because Projective Simulation (PS) is a RL model. So take a look on this illustration:
<br>
<br>
<br>

<img src="images/agent_env_interaction.png" width= "400"/>

You see the typical interaction between an agent and the environment. In our case the environment is the Ion Trap Environment (see Tutorial). The agent performs an action on the environment according to his observation and receives a possible reward. In the following we want to build a PS agent who is able to maximally entangle $N$ ions. So let's talk a bit about PS: <br>
<br>
Please do me a favor and read at least the abstract of this paper: __[link](https://arxiv.org/pdf/1104.3787.pdf)__ <br>
The following sentence is important: 
>"Projective simulation is based on a random walk through a network of clips,
which are elementary patches of episodic memory"

The brain of the agent is the so-called episodic compositional memory (ECM), which contains some network of clips. The clips are some instances of memory. Here you see an example:
<img src="images/general_clips_structure.png" width="300"/>
 
If the agent makes an observation an so called percept clip (blue) is activated or generated and based on the structure of the clip network and the weights of the so called edges (blue arrows) the agent chooses an action clip (red) and executes an action. This is called random walk. <br>
<br>
Here we do a so called action encoding. Therefore every percept clip gets its own decision tree and there is no connection between different percept clips.

Now that you understand the principal of PS let's start with the implementation! The mathematics will be introduced step by step.

## Implementation

Later we need something which represents the clip network with the clips, the edges and the weights of the edges. Therefore an adjencency matrix is very usefull. I have an example for you:

<img src="images/adj_matrix_example.png" width="500"/>
<br>
A field from the adjancency matrix represents the weight of a certain edge, which connects two clips. The edge points from one clip (row index) to another clip (column index). The agent is learning by adjusting the weights of the edges. In the beginning all weights are set to one. Later on we will see how the weights are changed.<br>
<br>

### The agents sceleton

Now we want to build a skeleton of our agent. <br>
The agent should have the ability to choose an action (here the id of the action) for a given observation and it should learn from the recieved reward. Therefore it needs an ECM, the actions and the decision tree structure (adjancency matrix with weights set to one). <br>
__TASK__: Write a new class skeleton `Agent()` with the `__init__()` function and two methods.


__SOLUTION__: <br>
This is a possible solution for an agent skeleton. I have completed the docs for you :)


In [4]:
class UniversalAgent(object):
    def __init__(self, ECM, actions, adj_matrix):
        """
            Projective Simulation Agent. Any typical ECM can be used.

            Args:
                ECM (object): Episodic compositional memory (ECM). The brain of the agent.
                actions (np.ndarray): An array of possible actions. Specified by the environment.
                adj_matrix (np.ndarray): Adjancency matrix representing the structure of the default decision tree.
            """

        self.ECM = ECM
        self.adj_matrix = adj_matrix
        self.actions = actions

    def step(self, observation):
        """
        Given an observation, returns the id of an action clip.

        Args:
            observation (object): The observation in some form of encoding.
        Returns:
            actionClip_id (int): The id of the chosen action clip.
        """

        return actionClip_id

    def learn(self, reward):
        """
        Given a reward, updates adjancency matrix and g matrix. See documentation of the ECM for the math.
        Args:
            reward (float): The received reward.
        """

### some functionality for the agent

Now we have to think about the functionality of the agent. Let's look at the `step()` method. Now i will explain, what the agent needs to do there and you will try to implement the expained methods from the ECM afterwards. <br>
(1) The agent accesses the ECM in order to either create a new percept clip for the observation or find an existing one, which was created before. If a new one was created it needs the adjancency matrix. 
(2) The agent performs a random walk through the decision tree of this percept clip and dicides for an action. Retruns the id of this action.

Here the completed `step()` method:

In [2]:
    def step(self, observation):
        """
        Given an observation, returns the id of an action clip.

        Args:
            observation (object): The observation in some form of encoding.
        Returns:
            actionClip_id (int): The id of the chosen action clip.
        """

        self.activeClip = self.ECM.get_or_create_percept_clip(observation, self.adj_matrix)
        actionClip_id = self.ECM.random_walk(self.activeClip)

        return actionClip_id

The `learn()` function is very easy. Here the agent just accesses the learn method from the ECM:

In [5]:
    def learn(self, reward):
        """
        Given a reward, updates adjancency matrix and g matrix. See documentation of the ECM for the math.
        Args:
            reward (float): The received reward.
        """
        self.ECM.learn(reward)

### CODE agent

Nice! We have a full functioning agent. A very silly agent, because the brain (ECM) is not implemented so far.

In [6]:
class UniversalAgent(object):
    def __init__(self, ECM, actions, adj_matrix):
        """
            Projective Simulation Agent. Any typical ECM can be used.

            Args:
                ECM (object): Episodic compositional memory (ECM). The brain of the agent.
                actions (np.ndarray): An array of possible actions. Specified by the environment.
                adj_matrix (np.ndarray): Adjancency matrix representing the structure of the default decision tree.
            """

        self.ECM = ECM
        self.adj_matrix = adj_matrix
        self.actions = actions

    def step(self, observation):
        """
        Given an observation, returns the id of an action clip.

        Args:
            observation (object): The observation in some form of encoding.
        Returns:
            actionClip_id (int): The id of the chosen action clip.
        """

        self.activeClip = self.ECM.get_or_create_percept_clip(observation, self.adj_matrix)
        actionClip_id = self.ECM.random_walk(self.activeClip)

        return actionClip_id

    def learn(self, reward):
        """
        Given a reward, updates adjancency matrix and g matrix.
        Args:
            reward (float): The received reward.
        """
        self.ECM.learn(reward)

### The brain of the agent (ECM)

In the ECM the percept and action clips are created and stored.  We use a dictionary with following form:
> percept_dict: {percept1 (str) : percept_clip1 (object), percept2 (str) : percept_clip2 (object), ...} <br>
> action_dict: {action1 (str) : action_clip1 (object), action2 (str) : action_clip2 (object),...}

We see two things: First the percept is stored as string, so we have to preprosses it somewhere. Secondly the percept and action clips are objects, so we need classes for the clips. <br>
__TASK__:  <br>
- Implement a parent class `Clip()` with instance attributes `self.id` and `self.type = "Clip"`
- Implement the classes `ActionClip(Clip)` and `PerceptClip(Clip)`. They should inherit the id from the `Clip()` class.
- Each clip needs an id and a label and the type. The label is just the percept (str) or the action (str). A percept clip needs also the adjancency matrix and a g-matrix (explained later). 

__SOLUTION__:

In [1]:
class Clip():
    """a clip"""
    def __init__(self, id):
        self.id = id
        self.type = "Clip"


class ActionClip(Clip):
    """an action clip"""
    def __init__(self, id, action):
        Clip.__init__(self, id)
        self.label = action
        self.type = "ActionClip"


class PerceptClip(Clip):
    """a percept clip"""
    def __init__(self, id, percept, adj_matrix, g_matrix):
        Clip.__init__(self, id)
        self.label = percept
        self.adj_matrix = adj_matrix
        self.g_matrix = g_matrix
        self.type = "PerceptClip"

Please take a look in the step and learn function of the agent. You probably see, that we should implement three functions from the ECM: <br>
- ECM.get_or_create_percept_clip()
- ECM.random_walk()
- ECM.learn()

Now i will provide you the ECM skeleton, but it is your job to give it life later on. You will find the three functions in the code. <br>
__TASK__: Read the docs carefully to get an overview of the ECM

In [2]:
### do not run ###

from ecm.Clips import ActionClip, PerceptClip
import numpy as np


class UniversalECM(object):
    def __init__(self, gamma_damping=0., eta_glow_damping=0.1, beta=1):
    
        self.num_actions = 0
        self.num_percepts = 0
        self.gamma_damping = gamma_damping
        self.eta_glow_damping = eta_glow_damping
        self.beta = beta

        self.percept_dict = {}  # stores the percept_clips with the corresponding percept, here: {percept (str) : Percept_clip (object), ...}
        self.action_dict = {}  # stores the action_clips with the corresponding number, here: {action (str) : Action_clip (object), ...}
        

    def get_action_dict(self, actions):
        """
        Initializes the action dictionary.
        It has this form: {action1 (str) : Action_clip1 (object), action2 (str) : Action_clip2 (object),...}

        Args:
            actions (array): [action1 (str), action2 (str), ...]
        """

    def get_or_create_percept_clip(self, observation, adj_matrix):
        """
        Finds the corresponding percept clip to the observation or creates a new percept clip for the observation.
        (1) Preprocesses the observation to a percept.
        (2) Tries to find the corresponding percept clip to this percept; returns the clip.
        (3) Creates a new percept clip if the percept has not been encountered before.
        (4) Sets the decision tree (in form of the adjancency matrix) and the g matrix as attributes of the percept clip.

        Args:
            observation (object): The observation in some form of encoding.
            adj_matrix (np.ndarray): The adjancency matrix representing the decision tree
        Returns:
            percept_clip (object): percept clip
        """

    def random_step(self, from_clip_index, adj_matrix, g_matrix):
        """
        Does a random step (transition) from a clip (from_clip_index) to a connected clip (to_clip_index).
        The connected clips are weighted with the softmax distribution.

        Args:
            from_clip_index (int): from this clip a random step will be performed
            adj_matrix (np.ndarray): the adjancency matrix of the percept clip from which the random walk is performed
            g_matrix (np.ndarray): the  g matrix of the percept clip from which the random walk is performed
        Returns:
            to_clip_index (int): probabilistically picked clip
            finished (boolean): True, if na action clip is found
        """

        return to_clip_index, finished

    def random_walk(self, percept_clip):
        """
        Performs a random walk through the decision tree from the percept clip
        (encoded in the adjancency matrix) to an action clip. Returns the id (number) of this action clip.

        Args:
            percept_clip (object): percept clip
        Returns:
            action_clip_index (int): Id (number) of the found action clip
        """

        return action_clip_index

    def learn(self, reward):
        """
        Updates the h_values of all edges according to the reward.
        Sets all g values to zero.

        Args:
            reward (float): received reward
        """
                
                
### helper functions ###
    
    def softmax(self, beta, h_values):
        """
        Calculates probabilities according to the softmax distribution.

        Args:
            beta (float): softmax parameter beta
            h_values (np.ndarray): array of the h-values
        Returns:
            tuple of probabilities, same order as the h_values
        """
        return ex / sum(ex)

    def preprocess(self, observation):
        """
        Given an observation, returns a percept.
        This function is just to emphasize the difference between observations
        issued by the environment and percepts which describe the observations
        as perceived by the agent.

        Args:
            observation (object): The observation in some form of encoding.
        Returns:
            percept (str): The observation encoded as a percept.
        """
        percept = str(observation)
        return percept


ModuleNotFoundError: No module named 'ecm'

Before we implement the three important functions, we should take a look on the `__init__()` function: 

In [ ]:
def __init__(self, gamma_damping=0., eta_glow_damping=0.1, beta=1):
    
    self.num_actions = 0
    self.num_percepts = 0
    self.gamma_damping = gamma_damping
    self.eta_glow_damping = eta_glow_damping
    self.beta = beta

    self.percept_dict = {}  # stores the percept_clips with the corresponding percept, here: {percept (str) : Percept_clip (object), ...}
    self.action_dict = {}  # stores the action_clips with the corresponding number, here: {action (str) : Action_clip (object), ...}

Here `num_actions` and `num_percepts` stands for the number of the action or percept clips. Later we use it to give the clips an id. gamma_damping, eta_glow_damping and beta are parameteres you can later play with to get good results. I will explain them when we come to the learning function.

#### ECM.get_or_create_percept_clip()

Here the percept clips are created and stored. Every percept clip gets its own decision tree (adj_matrix) a so called glow matrix (g_matrix). You can think of it as the short time memory of the agent. I will explain this later in detail. <br>
Here are the docs again: <br>
> Finds the corresponding percept clip to the observation or creates a new percept clip for the observation. <br>
  (1) Preprocesses the observation to a percept. <br>
  (2) Tries to find the corresponding percept clip to this percept; returns the clip. <br>
  (3) Creates a new percept clip if the percept has not been encountered before. Sets the decision tree (in form of the adjancency matrix) and the g matrix as attributes of the percept clip. <br>
<br>

__TASK__: <br>
I will give you some explanation for the 3 points, and then it is your job to write the code. <br>
(1): use the helperfunction `preprocess()`<br>
(2): look at the form of the `percept_dict` where the clips are stored and write a condition.<br>
(3): initialize an matrix with just zeros with the size of the adj_matrix and call it g_matrix. Store the new clip with `id`, `label`, `adj_matrix` and `g_matrix` in the dictionary. Consider, that the you have to add one to the `num_percepts`, because the next percept clips has to get another id. <br>
<br>
__SOLUTION__: <br>
Your solution will probably differ from this in some way. Neverless it could be also correct. For later i recommend using my solution, because of compatibility reasons.

In [3]:
def get_or_create_percept_clip(self, observation, adj_matrix):
    """
    Finds the corresponding percept clip to the observation or creates a new percept clip for the observation.
    (1) Preprocesses the observation to a percept.
    (2) Tries to find the corresponding percept clip to this percept; returns the clip.
    (3) Creates a new percept clip if the percept has not been encountered before. 
        Sets the decision tree (in form of the adjancency matrix) and the g matrix as attributes of the percept clip.

    Args:
        observation (object): The observation in some form of encoding.
        adj_matrix (np.ndarray): The adjancency matrix representing the decision tree
    Returns:
        percept_clip (object): percept clip
    """

    percept = self.preprocess(observation)

    if percept in self.percept_dict.keys():
        percept_clip = self.percept_dict[percept]
        return percept_clip
    else:
        # initialize g matrix with empty np.array
        g_matrix = np.zeros((len(adj_matrix), len(adj_matrix)))

        percept_clip = PerceptClip(self.num_percepts, percept, adj_matrix, g_matrix)
        self.percept_dict[percept] = percept_clip
        self.num_percepts += 1

        return percept_clip

#### ECM.random_walk()

Finally we came to a point where we can't ignore the math anymore. As previously explained, the agent chooses an action with a random walk through the decision tree. Each time the agent has to decide between two or more clips, it does a probabilistic choice due to the weights on the edges of the clips. The probabilities are calculated according to the so-called softmax distribution: <br>
<br>
$p_{ij}^{(t)} = p^{(t)}(c_i \rightarrow c_j) = e^{\beta \tilde{h}_{ij}^{(t)}} / \sum_k e^{\beta \tilde{h}_{ik}^{(t)}}$ <br>
<br>
with<br>
<br>
$\tilde{h}_{ij}^{(t)} = h_{ij}^{(t)} - \max_{k \in \mathcal{N}_i} h_{ik}^{(t)}$<br>
<br>
$p_{ij}^{(t)}$ is the probabilitiy at time $(t)$, that the agent chooses the transition from clip $i$ to clip $j$. $h_{ij}$ is a so called h value (weight), which is the $ij$-th entry of the adjancency matrix. The second equation avoids computational errors due to large numbers and $\mathcal{N}_i$ is the set of all clips connected to clip $c_i$. <br>
$\beta$ is the so-called softmax parameter. Later you can play with this paramter to see its effects on the results. I don't want to spoil you now :)
<br> 
__TASK:__ <br>
Implement the math for the `softmax()` function according to the equations above and the docs.

In [ ]:
def softmax(self, beta, h_values):
        """
        Calculates probabilities according to the softmax distribution.

        Args:
            beta (float): softmax parameter beta
            h_values (np.ndarray): array of h-values (row of the adjancency matrix without zero entries)
        Returns:
            prob (np.ndarray): array of probabilities, same order as the h_values
        """
        # fill in
        
        return prob

__SOLUTION:__

In [ ]:
def softmax(self, beta, h_values):
        """
        Calculates probabilities according to the softmax distribution.

        Args:
            beta (float): softmax parameter beta
            h_values (np.ndarray): array of h-values (row of the adjancency matrix without zero entries)
        Returns:
            tuple of probabilities, same order as the h_values
        """
        ex = np.array(h_values, dtype=np.float64) * beta
        ex = ex - max(ex)
        ex = np.exp(ex)
        prob = ex / sum(ex)
        
        return prob

Great! The first important step for the random walk is done. Out of an array of weigths we can get the probablities. The next goal is to write a function which is able to perform a random step. The first random step is the the step from the percept clip to one of the connected ones. We now try to write a funktion which takes the index of one clip and returns the index of the choosen clip according to the weights. The funktion should also have an output indicating if an action clip is found. For the warm up i have a short task for you: <br>
__TASK:__ <br>
Write down any arbitrary adjanceny matrix (you can also use the one from above) and find some easy way to see if a clip is an action clip just from the entries of the matrix. <br>
<br>
Now we implement the whole function. Therefore i have to explain one more thing. You may asked yourself what the g matrix does exactly. The g matrix is something like a short time memory of the agent. Previously made steps are stored in this matrix. At the beginning this matrix is just filled with zeros and has the same size as the adjancency matrix. If a random step is done this step is stored in the g matrix. The corresponding entry for the chosen edge in the g matrix is set to one. <br>
Now its your turn! Following functions could be useful: `enumerate()`and `np.random.choice()`<br>
__TASK:__ <br>
Implement the `random_step()` with the help of the explanation and the docs. This task is a harder then the previous ones. But please try it at first by youself. It's no shame if you need the solution for this.

In [ ]:
def random_step(self, from_clip_index, adj_matrix, g_matrix):
    """
    Does a random step (transition) from a clip (from_clip_index) to a connected clip (to_clip_index).
    The connected clips are weighted with the softmax distribution.

    Args:
        from_clip_index (int): from this clip a random step will be performed
        adj_matrix (np.ndarray): the adjancency matrix of the percept clip from which the random walk is performed
        g_matrix (np.ndarray): the  g matrix of the percept clip from which the random walk is performed
    Returns:
        to_clip_index (int): probabilistically picked clip
        finished (boolean): True, if an action clip is found
    """
    return to_clip_index, finished

__SOLUTION:__

In [ ]:
    def random_step(self, from_clip_index, adj_matrix, g_matrix):
        """
        Does a random step (transition) from a clip (from_clip_index) to a connected clip (to_clip_index).
        The connected clips are weighted with the softmax distribution.

        Args:
            from_clip_index (int): from this clip a random step will be performed
            adj_matrix (np.ndarray): the adjancency matrix of the percept clip from which the random walk is performed
            g_matrix (np.ndarray): the  g matrix of the percept clip from which the random walk is performed
        Returns:
            to_clip_index (int): probabilistically picked clip
            finished (boolean): True, if an action clip is found
        """

        finished = False

        row = adj_matrix[from_clip_index]
        enum_row = list(enumerate(row))
        
        # filter out the zero entries as there is no edge
        filtered_row = [enum_row[i] for i in range(len(enum_row)) if enum_row[i][1] != 0]

        if len(filtered_row) != 0:  # if the row has now entry != 0 an action clip is found

            # connected_clips_indices: indices of the connected clips
            # connected_clips_probabilities: h values of the connected clips
            connected_clips_indices = [filtered_row[i][0] for i in range(len(filtered_row))]
            connected_clips_probabilities = np.array([filtered_row[i][1] for i in range(len(filtered_row))])

            # pick one clip of all connected clips randomly weighted with the softmax distribution
            prob = self.softmax(self.beta, connected_clips_probabilities
            to_clip_index = np.random.choice(connected_clips_indices, p=prob))


            # set g value of the clip transition to one
            x, y = from_clip_index, to_clip_index
            g_matrix[x, y] = 1

        else:
            to_clip_index = from_clip_index
            finished = True

        return to_clip_index, finished

Congratulations! The hard part is over. But the agent is still not able to do a random walk and to learn. So let's finally introduce the most important equations of Projective Simulation: <br>
<br>
$h_{ij}^{'} = h_{ij} - \gamma \cdot (h_{ij} -1) + g_{ij} \cdot \lambda$<br>
<br>
and
<br>
<br>
$g_{ij}^{'} = g_{ij} \cdot (1- \eta)$<br>
<br>

These equations determine how the h and g values are updated. The whole network is updated according to these equations every time a random step was made. $\gamma$ is the gamma damping parameter. Here we can set it to zero, because it is just important if the environment changes in time. The reward $\lambda$ is one if the agent found the maximal SRV (see Tutorial), zero otherwise. $\eta$ is the glow damping parameter. This parameter is important for problems with a delayed reward. That means that the agent does not recieve a reward after each step (f.e. gridworld environment). It needs to remember the sequence of steps which brougt it to the reward. With $\eta$ one can adjust the shorttime forgetfulness of the agent. <br>
Note, that the first equation to update the h values is just needed in the `learn()` function, because if the agent does not learn (the reward is zero) all h values stay the same.<br>
Here it is better to apply the second equation not after each random step, but after each random walk. So let's do this! I will give you the solution straight, because you are still here and for that i want to do you a favor :)<br>
Despite of that, please read the function carfully and try to understand it.

In [ ]:
    def random_walk(self, percept_clip):
        """
        Performs a random walk through the decision tree from the percept clip
        (encoded in the adjancency matrix) to an action clip. Returns the id (number) of this action clip.

        Args:
            percept_clip (object): percept clip
        Returns:
            action_clip_index (int): Id (number) of the found action clip
        """

        # check, if the input clip is a percept clip
        if percept_clip.type != "PerceptClip":
            print("first clip in random walk needs to be a percept_clip")

        # adjancency matrix g_matrix (np.ndarray) from the percept clip
        adj_matrix = percept_clip.adj_matrix
        g_matrix = percept_clip.g_matrix

        # all clips connected to a percept clip are in the first row
        from_clip_index = 0

        finished = False

        # do random steps until reaching a action clip
        while not finished:
            to_clip_index, finished = self.random_step(from_clip_index, adj_matrix, g_matrix)
            from_clip_index = to_clip_index

        # damp all g values from the whole network
        for clip in self.percept_dict.values():
            if clip != percept_clip:
                clip.g_matrix *= (1 - self.eta_glow_damping)

        # subtract one to get the action clip index, because the first clip is always the percept clip
        action_clip_index = to_clip_index - 1

        return action_clip_index

#### ECM.learn()

This is the last function we have to implement. The whole network should be updated according to the first equation explained above. <br>
__TASK:__ <br>
Fill in the code according to the explanation and the docs.

In [ ]:
    def learn(self, reward):
        """
        Updates the h_values of all edges according to the reward.
        Sets all g values to zero.

        Args:
            reward (float): received rewardThis
        """

__SOLUTION:__ <br>
I added the last two rows. Here the g matrix is set to zero again if the agent learnt. 

In [ ]:
    def learn(self, reward):
        """
        Updates the h_values of all edges according to the reward.
        Sets all g values to zero.

        Args:
            reward (float): received reward
        """
        if reward != 0.0:
            for clip in self.percept_dict.values():
                clip.adj_matrix = clip.adj_matrix + reward * clip.g_matrix

            for clip in self.percept_dict.values():
                clip.g_matrix *= 0

## CODE Agent

Congratulations! You made it. You have written a fully functioning PS Agent! Here the final code:

In [1]:
import numpy as np


class UniversalECM(object):
    def __init__(self, gamma_damping=0., eta_glow_damping=0.1, beta=1):
        """

        """

        self.num_actions = 0
        self.num_percepts = 0
        self.gamma_damping = gamma_damping
        self.eta_glow_damping = eta_glow_damping
        self.beta = beta

        self.percept_dict = {}  # stores the percept_clips with the corresponding percept, here: {percept (str) : Percept_clip (object), ...}
        self.action_dict = {}  # stores the action_clips with the corresponding number, here: {action (str) : Action_clip (object), ...}

    def get_action_dict(self, actions):
        """
        Initializes the action dictionary.
        It has this form: {action1 (str) : Action_clip1 (object), action2 (str) : Action_clip2 (object),...}

        Args:
            actions (array): [action1 (str), action2 (str), ...]
        """
        for action in actions:
            action_clip = ActionClip(self.num_actions, action)
            self.action_dict[action] = action_clip
            self.num_actions += 1

    def get_or_create_percept_clip(self, observation, adj_matrix):
        """
        Finds the corresponding percept clip to the observation or creates a new percept clip for the observation.
        (1) Preprocesses the observation to a percept.
        (2) Tries to find the corresponding percept clip to this percept; returns the clip.
        (3) Creates a new percept clip if the percept has not been encountered before.
        (4) Sets the decision tree (in form of the adjancency matrix) and the g matrix as attributes of the percept clip.

        Args:
            observation (object): The observation in some form of encoding.
            adj_matrix (np.ndarray): The adjancency matrix representing the decision tree
        Returns:
            percept_clip (object): percept clip
        """

        percept = self.preprocess(observation)

        if percept in self.percept_dict.keys():
            percept_clip = self.percept_dict[percept]
            return percept_clip
        else:
            # initialize g matrix with empty np.array
            g_matrix = np.zeros((len(adj_matrix), len(adj_matrix)))

            percept_clip = PerceptClip(self.num_percepts, percept, adj_matrix, g_matrix)
            self.percept_dict[percept] = percept_clip
            self.num_percepts += 1

            return percept_clip

    def get_percept_clip(self, percept):
        return self.percept_dict[percept]

    def get_action_clip(self, action):
        return self.action_dict[action]

    def random_step(self, from_clip_index, adj_matrix, g_matrix):
        """
        Does a random step (transition) from a clip (from_clip_index) to a connected clip (to_clip_index).
        The connected clips are weighted with the softmax distribution.

        Args:
            from_clip_index (int): from this clip a random step will be performed
            adj_matrix (np.ndarray): the adjancency matrix of the percept clip from which the random walk is performed
            g_matrix (np.ndarray): the  g matrix of the percept clip from which the random walk is performed
        Returns:
            to_clip_index (int): probabilistically picked clip
            finished (boolean): True, if an action clip is found
        """

        finished = False

        row = adj_matrix[from_clip_index]
        enum_row = list(enumerate(row))
        filtered_row = [enum_row[i] for i in range(len(enum_row)) if enum_row[i][1] != 0]

        if len(filtered_row) != 0:  # if the row has now entry != 0 an action clip is found

            # connected_clips_indices: indices of the connected clips
            # connected_clips_probabilities: h values of the connected clips
            connected_clips_indices = [filtered_row[i][0] for i in range(len(filtered_row))]
            connected_clips_probabilities = np.array([filtered_row[i][1] for i in range(len(filtered_row))])

            # pick one clip of all connected clips randomly weighted with the softmax distribution
            to_clip_index = np.random.choice(connected_clips_indices,
                                             p=self.softmax(self.beta, connected_clips_probabilities))

            # set g value of the clip transition to one
            x, y = from_clip_index, to_clip_index
            g_matrix[x, y] = 1

        else:
            to_clip_index = from_clip_index
            finished = True

        return to_clip_index, finished

    def random_walk(self, percept_clip):
        """
        Performs a random walk through the decision tree from the percept clip
        (encoded in the adjancency matrix) to an action clip. Returns the id (number) of this action clip.

        Args:
            percept_clip (object): percept clip
        Returns:
            action_clip_index (int): Id (number) of the found action clip
        """

        # check, if the input clip is a percept clip
        if percept_clip.type != "PerceptClip":
            print("first clip in random walk needs to be a percept_clip")

        # adjancency matrix g_matrix (np.ndarray) from the percept clip
        adj_matrix = percept_clip.adj_matrix
        g_matrix = percept_clip.g_matrix

        # all clips connected to a percept clip are in the first row
        from_clip_index = 0

        finished = False

        # do random steps until reaching a action clip
        while not finished:
            to_clip_index, finished = self.random_step(from_clip_index, adj_matrix, g_matrix)
            from_clip_index = to_clip_index

        # damp all g values from the whole network
        for clip in self.percept_dict.values():
            if clip != percept_clip:
                clip.g_matrix *= (1 - self.eta_glow_damping)

        # subtract one to get the action clip index, because the first clip is always the percept clip
        action_clip_index = to_clip_index - 1

        return action_clip_index

    def learn(self, reward):
        """
        Updates the h_values of all edges according to the reward.
        Sets all g values to zero.

        Args:
            reward (float): received reward
        """
        if reward != 0.0:
            for clip in self.percept_dict.values():
                clip.adj_matrix = clip.adj_matrix + reward * clip.g_matrix

            for clip in self.percept_dict.values():
                clip.g_matrix *= 0
                
    #### helper functions ####

    def softmax(self, beta, h_values):
        """
        Calculates probabilities according to the softmax distribution.

        Args:
            beta (float): softmax parameter beta
            h_values (np.ndarray): array of the h-values
        Returns:
            tuple of probabilities, same order as the h_values
        """
        ex = np.array(h_values, dtype=np.float64) * beta
        ex = ex - max(ex)
        ex = np.exp(ex)

        return ex / sum(ex)

    def preprocess(self, observation):
        """
        Given an observation, returns a percept.
        This function is just to emphasize the difference between observations
        issued by the environment and percepts which describe the observations
        as perceived by the agent.

        Args:
            observation (object): The observation in some form of encoding.
        Returns:
            percept (str): The observation encoded as a percept.
        """
        percept = str(observation)
        return percept

Note, that this code is the most basic version of PS. You can give a tree structure as input, but it can't change. You could improve the code by adjusting the tree structure during the process of learning to generalize over actions for example. <br>
If you are further interested in the theory of PS i recommend to read the following paper: <br>
- __[Projective simulation for artificial intelligence](https://arxiv.org/pdf/1104.3787.pdf)__
- __[Projective simulation with generalization](https://arxiv.org/pdf/1504.02247.pdf)__


Now it is time to play with the agent and the ion trap environment. You can try this by yourself or you do the third tutorial. See you there :)